### References:
[1] Maciejewski, Filip B., Jacob Biamonte, Stuart Hadfield, and Davide Venturelli. "[Improving quantum approximate optimization by noise-directed adaptive remapping.](https://arxiv.org/abs/2404.01412)" arXiv preprint arXiv:2404.01412 (2024).

[2] Maciejewski, Filip B., Bao G. Bach, Maxime Dupont, P. Aaron Lott, Bhuvanesh Sundar, David E. Bernal Neira, Ilya Safro, and Davide Venturelli. "[A multilevel approach for solving large-scale qubo problems with noisy hybrid quantum approximate optimization.](https://arxiv.org/abs/2408.07793)" In 2024 IEEE High Performance Extreme Computing Conference (HPEC), pp. 1-10. IEEE, 2024.

[3] Maciejewski, Filip B., Stuart Hadfield, Benjamin Hall, Mark Hodson, Maxime Dupont, Bram Evert, James Sud et al. "[Design and execution of quantum circuits using tens of superconducting qubits and thousands of gates for dense Ising optimization problems.](https://arxiv.org/abs/2308.12423)" Physical Review Applied 22, no. 4 (2024): 044074.

[4] Tam, Wai-Hong, Hiromichi Matsuyama, Ryo Sakai, and Yu Yamashiro. "[Enhancing NDAR with Delay-Gate-Induced Amplitude Damping.]"(https://arxiv.org/abs/2504.12628) arXiv preprint arXiv:2504.12628 (2025).

[5] Lykov, Danylo, Ruslan Shaydulin, Yue Sun, Yuri Alexeev, and Marco Pistoia. "[Fast simulation of high-depth qaoa circuits.](https://arxiv.org/abs/2309.04841)" In Proceedings of the SC'23 Workshops of The International Conference on High Performance Computing, Network, Storage, and Analysis, pp. 1443-1451. 2023.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from quapopt import AVAILABLE_SIMULATORS

os.makedirs('../temp', exist_ok=True)

### Generate a random Hamiltonian instance

* In this example, we generate a random Sherrington-Kirkpatrick Hamiltonian.
* In general, we can generate other classes, or read the instances from file.
* We can also solve the Hamiltonian classicaly, to get the ground state energy and the highest energy. (this, of course, should be done offline, before running the QAOA)



In [ ]:
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.data_analysis.data_handling import (COEFFICIENTS_TYPE,
                                                 COEFFICIENTS_DISTRIBUTION,
                                                 CoefficientsDistributionSpecifier,
                                                 HAMILTONIAN_MODELS)

number_of_qubits = 20
seed_cost_hamiltonian = 0

coefficients_type = COEFFICIENTS_TYPE.DISCRETE
coefficients_distribution = COEFFICIENTS_DISTRIBUTION.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

# We generate a Hamiltonian instance. In this case it's a random Sherrington-Kirkpatrick Hamiltonian
hamiltonian_model = HAMILTONIAN_MODELS.SherringtonKirkpatrick
localities = (1, 2)
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True)

print("Class description (cost):", cost_hamiltonian.hamiltonian_class_description)
print("Instance description (cost):", cost_hamiltonian.hamiltonian_instance_description)

if cost_hamiltonian.lowest_energy is None:
    print("SOLVING THE HAMILTONIAN CLASSICALLY")
    # if we wish, we can solve the Hamiltonian classically
    cost_hamiltonian.solve_hamiltonian(both_directions=True)

ground_state_energy = cost_hamiltonian.ground_state_energy
highest_energy = cost_hamiltonian.highest_energy

print(f"Ground state energy: {ground_state_energy}\nbackend: {type(cost_hamiltonian.couplings)}", )

### Bitflips and permutations

* In general, we can add bitflips and permutations to the Hamiltonian representation.
* Neither should change the optimization in noiseless setting, but they can affect it in noisy settings, see Refs.~[1,2,3,4] for examples of that in real-life experiments.
* Bitflip gauges in particular are used in Noise-Directed Adaptive Remapping (NDAR) [1,3,4].
* We observe that mapping the amplitude damping attractor state (|00...0>) to lower-energy state will improve the optimization.
* We now demonstrate this in noisy simulation with biased readout noise and three gauges:

    a) random gauge

    b) gauge that maps |00...0> to the lowest-energy state

    c) gauge that maps |00...0> to the highest-energy state

* In what follows, we will run p=1 QAOA for easy visualization.
*




In [ ]:
numpy_rng_sampling = np.random.default_rng(seed=0)
zero_bitstring = np.zeros(number_of_qubits, dtype=int)

known_states = cost_hamiltonian.get_known_energies_dict()
best_bitstring = known_states['lowest_energy_state']
worst_bitstring = known_states['highest_energy_state']

gauges_identifiers = ['random', 'best', 'worst']
bitstrings_ordered = [zero_bitstring, best_bitstring, worst_bitstring, ]
energies = cost_hamiltonian.evaluate_energy(bitstrings_array=np.array(bitstrings_ordered))

#IMPORTANT TO MAKE COPY BEFORE BITFLIP (because it's inplace operation)
hamiltonian_representations = [cost_hamiltonian.copy().apply_bitflip(bitflip_tuple=tuple(bts)) for bts in
                               bitstrings_ordered]

energies_zero_bts = [float(ham.evaluate_energy(bitstrings_array=np.array([zero_bitstring]))[0]) for ham in
                     hamiltonian_representations]
print('bitstrings energies before gauge transformation:', energies)
print('energies of |00...0> state after gauge transformation:', energies_zero_bts)



In [ ]:
#TODO(FBM): make imports easier
from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner
from quapopt.optimization.parameter_setting.non_adaptive_optimization.SimpleGridOptimizer import SimpleGridOptimizer
from quapopt.optimization.parameter_setting import ParametersBoundType as PBT
from quapopt.optimization.QAOA.implementation.QAOARunnerSampler import QAOARunnerSampler
from quapopt.optimization.QAOA.simulation.QAOARunnerExpValues import QAOARunnerExpValues
from quapopt.circuits.noise.simulation.ClassicalMeasurementNoiseSampler import (ClassicalMeasurementNoiseSampler,
                                                                                MeasurementNoiseType)
from quapopt.optimization.parameter_setting.variational.scipy_tools.ScipyOptimizerWrapped import ScipyOptimizerWrapped

#Number of objective function calls
number_of_function_calls = 200
#Number of measurements to estimate the expectation values
number_of_samples = 1000
#we can specify the details of classical optimizer here.
classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[(-np.pi, np.pi)] * 2,
                                            optimizer_name='COBYLA',
                                            optimizer_kwargs=None,
                                            basinhopping=True,
                                            basinhopping_kwargs={'niter': 3},
                                            starting_point=[0.05] * 2
                                            )
#Here we define the noise model properties.
#In this example, it's just amplitude damping added at the end of the circuit, exactly the same for all qubits.
#Fully asymmetric noise -- equivalent to amplitude damping at the end of the circuit
p_01 = 0.5
p_10 = None

#set up the grid optimizer
#we will evaluate points on a grid
grid_size_total = 10 ** 4
grid_sampler = SimpleGridOptimizer(parameter_bounds=[(PBT.RANGE, (-np.pi, np.pi))] * 2,
                                   max_trials=grid_size_total)


In [ ]:
all_results_optimizer = {}
all_results_grids = {}
for bitstring_name, hamiltonian in tqdm(list(zip(gauges_identifiers[0:], hamiltonian_representations[0:])), position=0,
                                        colour='blue'):

    #Defininig the noise model
    CMNS = ClassicalMeasurementNoiseSampler(noise_type=MeasurementNoiseType.TP_1q_identical,
                                            noise_description={'p_01': p_01,
                                                               'p_10': p_10})
    #This calculates effective Hamiltonian representation with noise model for exact exp. value calculations
    #(for simulation only)
    CMNS.add_noisy_hamiltonian_representations(hamiltonian_representations_dict={0: hamiltonian})

    for noise_string, noise_model in list(zip(['Ideal', 'Noisy'][0:], [None, CMNS][0:])):
        qaoa_sampler = QAOARunnerSampler(
            hamiltonian_representations_cost=[hamiltonian],
            store_n_best_results=1,
            store_full_information_in_history=False,
            numpy_rng_sampling=numpy_rng_sampling)

        if 'cuda' in AVAILABLE_SIMULATORS:
            #If we have GPU, we use qokit because it's faster than qiskit. See Ref. [5] for details.
            qaoa_sampler.initialize_backend_qokit(qokit_backend='gpu')
        else:
            qaoa_sampler.initialize_backend_qiskit(qaoa_depth=1)

        #Run noisy sampling QAOA optimization.
        qaoa_optimizer = QAOAOptimizationRunner(qaoa_runner=qaoa_sampler)
        _, opt_results = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                         number_of_function_calls=number_of_function_calls,
                                                         classical_optimizer=classical_optimizer,
                                                         measurement_noise=noise_model,
                                                         number_of_samples=number_of_samples,
                                                         verbosity=0,
                                                         show_progress_bar=False,
                                                         optimizer_seed=42)
        all_results_optimizer[f'{bitstring_name}_{noise_string}'] = opt_results

        #Run exact expected-values optimization
        qaoa_exp_values_simulator = QAOARunnerExpValues(
            hamiltonian_representations_cost=[hamiltonian],
            store_full_information_in_history=False,
            simulator_name=None)

        qaoa_optimizer_grid = QAOAOptimizationRunner(qaoa_runner=qaoa_exp_values_simulator)

        _, opt_results_grid = qaoa_optimizer_grid.run_optimization(qaoa_depth=1,
                                                                   number_of_function_calls=grid_size_total,
                                                                   classical_optimizer=grid_sampler,
                                                                   measurement_noise=noise_model,
                                                                   memory_intensive=True,
                                                                   verbosity=0,
                                                                   show_progress_bar=False)
        all_results_grids[f'{bitstring_name}_{noise_string}'] = opt_results_grid




### Visualization

* Let's see how different gauges change the noisy landscapes.

In [ ]:
import plotly
from quapopt.data_analysis.visualization import optimization_visualization as opt_vis
from quapopt.data_analysis.data_handling import STANDARD_NAMES_VARIABLES as SNV

plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

x_name = f"{SNV.Angles.id_long}-0"
y_name = f"{SNV.Angles.id_long}-1"
fom_name = f"{SNV.EnergyMean.id_long}"
names_map = {f"ARG-0": f"{SNV.Angles.id_long}-0",
             f"ARG-1": f"{SNV.Angles.id_long}-1",
             f"FunctionValue": f"{SNV.EnergyMean.id_long}"}

In [ ]:
print(all_results_grids.keys())
print(all_results_grids['best_Ideal'].trials_dataframe.sort_values(['FunctionValue']))
print(all_results_grids['best_Noisy'].trials_dataframe.sort_values(['FunctionValue']))

In [ ]:
#this can made it slower than just opening in browser, but it allows to show the plots in Jupyter cells
show_plots_in_jupyter_cell = True

for plot_in_3d in [False, True]:
    titles = list(all_results_grids.keys())
    dataframes_to_plot = []
    heatmaps_grid, scatters_grid = [], []
    for key in titles:
        res = all_results_optimizer[key]
        df = res.trials_dataframe
        df.rename(columns=names_map, inplace=True)

        res_grid = all_results_grids[key]
        df_grid = res_grid.trials_dataframe
        df_grid.rename(columns=names_map, inplace=True)

        dataframes_to_plot.append(df)
        heatmap_grid, scatter_grid = opt_vis.get_interpolated_heatmap(df=df_grid,
                                                                      x_name=x_name,
                                                                      y_name=y_name,
                                                                      fom_name=fom_name,
                                                                      bounds_x=(-np.pi, np.pi),
                                                                      bounds_y=(-np.pi, np.pi),
                                                                      in_3d=plot_in_3d
                                                                      )
        heatmaps_grid.append(heatmap_grid)
        scatters_grid.append(scatter_grid)

    trajectory_fig = opt_vis.plot_multiple_heatmaps(dataframes=dataframes_to_plot,
                                                    x_name=x_name,
                                                    y_name=y_name,
                                                    fom_name=fom_name,
                                                    bounds_x=(-np.pi, np.pi),
                                                    bounds_y=(-np.pi, np.pi),
                                                    heatmaps_input=heatmaps_grid,
                                                    scatter_inputs=scatters_grid,
                                                    add_trajectories=True,
                                                    titles=titles,
                                                    global_normalization=True,
                                                    colormap_name='Viridis_r',
                                                    minimization=True,
                                                    in_3d=plot_in_3d)

    plotly.offline.plot(trajectory_fig,
                        filename=f'../temp/trajectories_gauges_3d-{plot_in_3d}.html')
    if not show_plots_in_jupyter_cell:
        continue
    trajectory_fig.show()


* We observe that uncorrelated, identical biased readout noise changes the landscape somewhat trivially -- it flattens it. However, by changing the gauge, we can change the absolute values of the measured energy significantly.
* In real experiments, the noise is much more complex, but it has been observed that similar correlation between optimization quality and the energy of the |00...0> state can be observed [1,2,4].



### Multiple random gauges and dependence on the energy of the |00...0> state

* We can investigate the dependence between optimization quality and the energy of the |00...0> on the gauge-transformed Hamiltonian, similarly as was done in Ref. [1] (see Figure 2 in the paper).
* To this aim, we will generate multiple random gauges and optimize the QAOA for each gauge-transformed Hamiltonian


In [ ]:
#Generate random gauges
number_of_gauges = 30
random_bitstrings = numpy_rng_sampling.integers(0, 2, size=(number_of_gauges, number_of_qubits))
zero_energies = cost_hamiltonian.evaluate_energy(bitstrings_array=random_bitstrings)

number_of_samples = 1000
#Small number of function calls so it's not too slow. In real-life applications, we would use more.
number_of_function_calls = 50
CMNS = ClassicalMeasurementNoiseSampler(noise_type=MeasurementNoiseType.TP_1q_identical,
                                        noise_description={'p_01': p_01,
                                                           'p_10': p_10})

optimization_results = []
best_function_values = []
x_name_gauges, y_name_gauges = 'energy of |0..0>', 'Optimized Energy'
for bts_gauge, zero_energy in tqdm(list(zip(random_bitstrings, zero_energies)), position=0, colour='blue'):
    hamiltonian = cost_hamiltonian.copy().apply_bitflip(bitflip_tuple=tuple(bts_gauge))
    qaoa_sampler = QAOARunnerSampler(
        hamiltonian_representations_cost=[hamiltonian],
        hamiltonian_representations_phase=None,
        store_n_best_results=1,
        store_full_information_in_history=False,
        numpy_rng_sampling=numpy_rng_sampling)

    if 'cuda' in AVAILABLE_SIMULATORS:
        #If we have GPU, we use qokit for demonstration because it's faster than qiskit
        qaoa_sampler.initialize_backend_qokit(qokit_backend='gpu')
    else:
        qaoa_sampler.initialize_backend_qiskit(qaoa_depth=1)

    qaoa_optimizer = QAOAOptimizationRunner(qaoa_runner=qaoa_sampler)

    best_result, opt_results = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                               number_of_function_calls=number_of_function_calls,
                                                               classical_optimizer=classical_optimizer,
                                                               measurement_noise=CMNS,
                                                               number_of_samples=number_of_samples,
                                                               verbosity=0,
                                                               show_progress_bar=False,
                                                               optimizer_seed=42)
    optimization_results.append(opt_results)

    best_value = opt_results.best_value
    best_function_values.append(pd.DataFrame({y_name_gauges: [float(best_value)],
                                              x_name_gauges: [float(zero_energy)]}))



#TODO(FBM): add correlation analysis to this example


### Visualization

* Let's see what is the relationship between the effective energy of the |00...0> state and the optimized energy.

In [ ]:

df_results_grid = pd.concat(best_function_values, ignore_index=True)
figure_grid_histogram = opt_vis.plot_grid_histogram(df=df_results_grid,
                                                    x_name=x_name_gauges,
                                                    y_name=y_name_gauges,
                                                    x_bins=10,
                                                    y_bins=10,
                                                    )

plotly.offline.plot(figure_grid_histogram,
                    filename=f'../temp/gauges_grid.html')

if show_plots_in_jupyter_cell:
    figure_grid_histogram.show()
